In [1]:
!pip install numpy pandas scikit-learn joblib fastapi uvicorn

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 3.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.1 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import joblib
import os

In [ ]:
import pandas as pd
import numpy as np
import time

# Generar 1000 puntos de datos (1000 minutos ~ casi 17 horas)
num_points = 1000
timestamps = np.arange(int(time.time()) - num_points * 60, int(time.time()), 60)
prices = 27000 + np.cumsum(np.random.randn(num_points))  # Precio base + ruido aleatorio

# Crear el DataFrame
data = pd.DataFrame({
    'timestamp': timestamps,
    'price': prices
})

# Guardarlo como CSV
data.to_csv('price_history_sample.csv', index=False)

print("✅ Archivo 'price_history_sample.csv' generado exitosamente.")


In [ ]:
# 1. Cargar los datos históricos
data = pd.read_csv('price_history_sample.csv')  # Archivo de entrenamiento

In [ ]:
# 2. Generar features
# (Por ejemplo: diferencias de precios consecutivas)
data['delta_1'] = data['price'].diff(1)
data['delta_5'] = data['price'].diff(5)
data['delta_10'] = data['price'].diff(10)
data['delta_30'] = data['price'].diff(30)

In [ ]:
# 3. Target: % de variación en la próxima hora
data['future_change'] = data['price'].shift(-60)  # 60 minutos adelante
data['future_pct_change'] = ((data['future_change'] - data['price']) / data['price']) * 100

In [ ]:
# 4. Limpiar NaNs
data = data.dropna()

In [ ]:
# 5. Separar variables
X = data[['delta_1', 'delta_5', 'delta_10', 'delta_30']]
y = data['future_pct_change']

In [ ]:
# 6. Split en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 7. Entrenar el modelo
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

In [ ]:
# 8. Validar
y_pred = model.predict(X_val)
print("MAE:", mean_absolute_error(y_val, y_pred))

In [ ]:
os.makedirs('model_training', exist_ok=True)

joblib.dump(model, 'model_training/model_gb.pkl')

In [ ]:
!pip install skl2onnx onnx

In [ ]:
import joblib
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

In [ ]:
# 1. Cargar modelo entrenado
model = joblib.load('model_training/model_gb.pkl')

In [ ]:
# 2. Definir la forma de entrada
# Suponemos que la entrada tiene 4 características: delta_1, delta_5, delta_10, delta_30
initial_type = [('float_input', FloatTensorType([None, 4]))]

In [ ]:
# 3. Convertir a ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

In [ ]:
# 4. Guardar el modelo ONNX
with open('model_training/model_gb.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

print("✅ Conversión completada: model_gb.onnx generado.")